In [ ]:
import serial
import time
import openai
import os
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split 
import time
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import leastsq
from matplotlib.animation import FuncAnimation

# 基础起点
x_A11_0, y_A11_0,z_A11_0=[0,0,0]
x_B11_0, y_B11_0,z_B11_0=[70,0,0]
x_C11_0, y_C11_0,z_C11_0=[35.0, 66.332495807108, 0]
x_A12_0, y_A12_0,z_A12_0=[108.75157316,4.10575494,72.06231241]
x_A21_0, y_A21_0,z_A21_0=[-35.48529129,41.51477486,43.30259276]
x_A22_0, y_A22_0,z_A22_0=[80.66406895,47.1349648,105.29052639]
x_B12_0, y_B12_0,z_B12_0=[23.311178,96.52786769,74.88537849]
x_B21_0, y_B21_0,z_B21_0=[52.80556607,-37.72076529,30.45335499]
x_B22_0, y_B22_0,z_B22_0=[-3.18930982, 37.10745627,123.11681462]
x_C12_0, y_C12_0,z_C12_0=[-20.21163927,-26.2027968,74.9372951]
x_C21_0, y_C21_0,z_C21_0=[81.37150914,66.07597302,26.18347404]
x_C22_0, y_C22_0,z_C22_0=[51.75832892,-28.62328536,111.72796897]

# 定义原始传感器长度 (mm)
# 节点 A11 
L_A11B11_0,L_A11C11_0,L_A11B21_0,L_A11C12_0 =[75,75,70,80]
# 节点 A12 
L_A12B11_0,L_A12B21_0,L_A12C21_0, L_A12C22_0 =[80,80,80,75]
# 节点 A21 
L_A21B12_0,L_A21B22_0,L_A21C11_0,L_A21C12_0 =[85,85,85,75]
# 节点 A22 
L_A22B12_0,L_A22B22_0,L_A22C21_0,L_A22C22_0=[80,85,80,80]
# 节点 B11 
L_B11C11_0,L_B11C21_0 =[70,70]
# 节点 B12 
L_B12C11_0,L_B12C21_0 =[80,80]
# 节点 B21 
L_B21C12_0,L_B21C22_0 =[85,80]
# 节点 B22
L_B22C12_0,L_B22C22_0 =[80,85]

# 初始值获取 MΩ
R=[2.31,2.95,1.61,
   2.68,4.30,3.55,
   0.58,3.22,4.35,
   3.82,4.44,2.85,
   1.39,3.52,1.04,
   2.80,8.44,2.83,
   2.23,3.15,0.60,
   3.33,1.02,2.78]

R_B21C12_0 = R[0]
R_A11C12_0 = R[1]
R_A11B21_0 = R[2]

R_B22C12_0 = R[3]
R_A21B22_0 = R[4]
R_A21C12_0 = R[5]

R_A21C11_0 = R[6]
R_A21B12_0 = R[7]
R_B12C11_0 = R[8] 
              
R_A22C22_0 = R[9]                 
R_A22B22_0 = R[10]
R_B22C22_0 = R[11]

R_A22B12_0= R[12]   
R_A22C21_0= R[13] 
R_B12C21_0= R[14] 

# 第二个
R_A12C21_0= R[15]  
R_A12B11_0= R[16]  
R_B11C21_0= R[17]  

R_A12B21_0= R[18]  
R_A12C22_0= R[19]  
R_B21C22_0= R[20]  

R_A11B11_0= R[21]  
R_A11C11_0= R[22]  
R_B11C11_0= R[23]  

# 定义方程组
def update():
    ax.clear()  # 清除原有的绘图内容
    # 定义原点
    x_A11, y_A11,z_A11=[0,0,0]
    x_B11, y_B11,z_B11=[70,0,0]
    x_C11, y_C11,z_C11=[35.0,66.332495807108, 0]
    # 求解方程式
    def equations(x):
        # 定义各个节点的坐标值
        x_A12, y_A12, z_A12, x_A21, y_A21, z_A21, x_A22, y_A22, z_A22, x_B12, y_B12, z_B12, x_B21, y_B21, z_B21,x_B22, y_B22, z_B22, x_C12, y_C12, z_C12, x_C21, y_C21, z_C21, x_C22, y_C22, z_C22 = x
            
        # 定义杆和索的长度 mm
        rod_length = 135
    
        # 定义方程组 杆的长度
        # A11 A12
        eq1 = np.sqrt((x_A12 - x_A11)**2 + (y_A12 - y_A11)**2 +(z_A12 - z_A11)**2) - rod_length
        # A21 A22
        eq2 = np.sqrt((x_A22 - x_A21)**2 + (y_A22 - y_A21)**2 +(z_A22 - z_A21)**2) - rod_length
        # B11 B12
        eq3 = np.sqrt((x_B12 - x_B11)**2 + (y_B12 - y_B11)**2 +(z_B12 - z_B11)**2) - rod_length
        # B21 B22
        eq4 = np.sqrt((x_B22 - x_B21)**2 + (y_B22 - y_B21)**2 +(z_B22 - z_B21)**2) - rod_length
        # C11 C12
        eq5 = np.sqrt((x_C12 - x_C11)**2 + (y_C12 - y_C11)**2 +(z_C12 - z_C11)**2) - rod_length
        # C21 C22
        eq6 = np.sqrt((x_C22 - x_C21)**2 + (y_C22 - y_C21)**2 +(z_C22 - z_C21)**2) - rod_length
        
        # 定义索的长度 cable_length =L
        # 节点 A11 和方程
        # L_A11B11 L_A11C11
        # L_A11B21 = 70
        # L_A11C12 = 80
        eq7 = np.sqrt((x_B21 - x_A11)**2 + (y_B21 - y_A11)**2 +(z_B21 - z_A11)**2) - L_A11B21
        eq8 = np.sqrt((x_C12 - x_A11)**2 + (y_C12 - y_A11)**2 +(z_C12 - z_A11)**2) - L_A11C12
            
        # 节点 B11 和方程
        # L_B11C11 L_B11A11
        # L_B11A12 = 80
        # L_B11C21 = 70
        eq9 = np.sqrt((x_B11 - x_A12)**2 + (y_B11 - y_A12)**2 +(z_B11 - z_A12)**2) - L_A12B11
        eq10 =np.sqrt((x_B11 - x_C21)**2 + (y_B11 - y_C21)**2 +(z_B11 - z_C21)**2) - L_B11C21
        
        # 节点 C11 和方程
        # L_C11A11 L_C11B11
        # L_C11A21 = 85
        # L_C11B12 = 80
        eq11 = np.sqrt((x_C11 - x_A21)**2 + (y_C11 - y_A21)**2 +(z_C11 - z_A21)**2) - L_A21C11
        eq12 = np.sqrt((x_C11 - x_B12)**2 + (y_C11 - y_B12)**2 +(z_C11 - z_B12)**2) - L_B12C11
            
        #  节点 A12 和方程
        #  LA12B11
        # L_A12B21 = 80
        # L_A12C22 = 75
        # L_A12C21 = 80
        eq13 = np.sqrt((x_A12 - x_B21)**2 + (y_A12 - y_B21)**2 +(z_A12 - z_B21)**2) - L_A12B21
        eq14 = np.sqrt((x_A12 - x_C22)**2 + (y_A12 - y_C22)**2 +(z_A12 - z_C22)**2) - L_A12C22
        eq15 = np.sqrt((x_A12 - x_C21)**2 + (y_A12 - y_C21)**2 +(z_A12 - z_C21)**2) - L_A12C21
        
        # 节点 A21 和方程
        # L_A21C11  85  95  95
        # L_A21C12 = 75
        # L_A21B22 = 85
        # L_A21B12 = 85
        eq16 = np.sqrt((x_A21 - x_C12)**2 + (y_A21 - y_C12)**2 +(z_A21 - z_C12)**2) - L_A21C12
        eq17 = np.sqrt((x_A21 - x_B22)**2 + (y_A21 - y_B22)**2 +(z_A21 - z_B22)**2) - L_A21B22
        eq18 = np.sqrt((x_A21 - x_B12)**2 + (y_A21 - y_B12)**2 +(z_A21 - z_B12)**2) - L_A21B12
        
        # 节点 A22 和方程
        # L_A22C22 = 80
        # L_A22C21 = 80
        # L_A22B22 = 85
        # L_A22B12 = 80
        eq19 = np.sqrt((x_A22 - x_C22)**2 + (y_A22 - y_C22)**2 +(z_A22 - z_C22)**2) - L_A22C22
        eq20 = np.sqrt((x_A22 - x_C21)**2 + (y_A22 - y_C21)**2 +(z_A22 - z_C21)**2) - L_A22C21
        eq21 = np.sqrt((x_A22 - x_B22)**2 + (y_A22 - y_B22)**2 +(z_A22 - z_B22)**2) - L_A22B22
        eq22 = np.sqrt((x_A22 - x_B12)**2 + (y_A22 - y_B12)**2 +(z_A22 - z_B12)**2) - L_A22B12 
        
        # 节点 B12 和方程
        # L_B12A21 L_B12A22 L_B12C11  L_B12C21 =80
        # L_B12C21 =80
        eq23 = np.sqrt((x_B12 - x_C21)**2 + (y_B12 - y_C21)**2 +(z_B12 - z_C21)**2) - L_B12C21
        
        # 节点 B21 和方程
        # L_B21A11 L_B21A12 
        # L_B21C12 = 85
        # L_B21C22 = 80
        eq24 = np.sqrt((x_B21 - x_C12)**2 + (y_B21 - y_C12)**2 +(z_B21 - z_C12)**2) - L_B21C12
        eq25 = np.sqrt((x_B21 - x_C22)**2 + (y_B21 - y_C22)**2 +(z_B21 - z_C22)**2) - L_B21C22
        
        # 节点 B22 和方程
        # L_B22A21 L_B22A22 
        # L_B22C12 = 80
        # L_B22C22 = 85
        eq26 = np.sqrt((x_B22 - x_C12)**2 + (y_B22 - y_C12)**2 +(z_B22 - z_C12)**2) - L_B22C12
        eq27 = np.sqrt((x_B22 - x_C22)**2 + (y_B22 - y_C22)**2 +(z_B22 - z_C22)**2) - L_B22C22 
                
        # 返回方程组的结果
        return [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13, eq14, eq15, eq16, eq17,eq18, eq19, eq20, eq21, eq22, eq23, eq24, eq25, eq26, eq27]

    #  采用初始值作为参考
    x_guess = [108.75157386,  4.10573923,  72.06231265, -35.48529059,  41.51478065,
                43.30259659,  80.6640691,   47.13495761, 105.29053279,  23.31118531,
                96.52786728,  74.88538101,  52.80556015, -37.7207717,   30.45335623,
                -3.1893104,   37.10745251, 123.11681805, -20.21164493, -26.20279503,
                74.93729387,  81.37151866,  66.07596787,  26.18348154,  51.7583249,
                -28.62329171, 111.72797017]
    # 使用leastsq函数求解方程组
    solution, _ = leastsq(equations, x_guess)
    # x_guess = solution

    # 标记坐标点的位置
    x_A12 = solution[0]
    y_A12 = solution[1]
    z_A12 = solution[2]
    x_A21 = solution[3]
    y_A21 = solution[4]
    z_A21 = solution[5]
    x_A22 = solution[6]
    y_A22 = solution[7]
    z_A22 = solution[8]
    x_B12 = solution[9]
    y_B12 = solution[10]
    z_B12 = solution[11]
    x_B21 = solution[12]
    y_B21 = solution[13]
    z_B21 = solution[14]
    x_B22 = solution[15]
    y_B22 = solution[16]
    z_B22 = solution[17]
    x_C12 = solution[18]
    y_C12 = solution[19]
    z_C12 = solution[20]
    x_C21 = solution[21]
    y_C21 = solution[22]
    z_C21 = solution[23]
    x_C22 = solution[24]
    y_C22 = solution[25]
    z_C22 = solution[26]
       
    new_nodes = np.array([
        # A11 B11 C11
        [x_A11, y_A11, z_A11],  
        [x_B11, y_B11, z_B11],
        [x_C11, y_C11, z_C11],
        # A12 B12 C12
        [x_A12, y_A12, z_A12],
        [x_B12, y_B12, z_B12],
        [x_C12, y_C12, z_C12],
        #  A21 B21 C21
        [x_A21, y_A21, z_A21],
        [x_B21, y_B21, z_B21],
        [x_C21, y_C21, z_C21],
        # A22 B22 C22
        [x_A22, y_A22, z_A22],
        [x_B22, y_B22, z_B22],
        [x_C22, y_C22, z_C22],
        ])
    
    # 定义truss
    truss = np.array([
        #  truss A11& A12
        [0, 3],
        #  truss A21 & A22
        [6, 9],
        #  truss B11 & B12
        [1, 4],
        #  truss B21 & B22
        [7, 10],
        #  truss C11 & C12
        [2, 5],
        #  truss C21 & C22
        [8, 11]        
    ])
            
    # 定义节点名称
    cable = np.array([ 
        # 共计24根cables
        # A11 =0，B11=1, B21=7, C11=2, C12=5
        [0, 1],
        [0, 7],
        [0, 2],
        [0, 5],
        # A12=3 , B11=1, B21 =7, C21=8, C22=11
        [3, 1],
        [3, 7],
        [3, 8],
        [3, 11],
        # A21=6, C11=2, C12=5, B12=4, B22=10
        [6, 2],
        [6, 5],
        [6, 4],
        [6, 10],
        # A22=9, B12=4, B22=10, C21=8, C22=11
        [9, 4],
        [9, 10],
        [9, 8],
        [9, 11],
        # B11=1 A11=0 C11=2 C21=8 A12=3
        [1, 2],
        [1, 8],
        # B12=4 A12=3 C11=2 C21=8 A22=9
        [4, 2],
        [4, 8],
        # B21=7 A11=0 C12=5 A12=3 C22=11
        [7, 5],
        [7, 11],
        # B22=10 A11=0 C12=5 A12=3 C22=11
        [10, 5],
        [10, 11],
    ])

    # 定义节点名称
    node_names = ['A11', 'B11', 'C11', 'A12', 'B12', 'C12', 'A21', 'B21', 'C21','A22', 'B22', 'C22']

    # 绘制节点
    ax.scatter(new_nodes[:, 0], new_nodes[:, 1], new_nodes[:, 2], color='red', s=200) 
    # 绘制truss 
    for truss in truss:
        x = [new_nodes[truss[0], 0], new_nodes[truss[1], 0]]
        y = [new_nodes[truss[0], 1], new_nodes[truss[1], 1]]
        z = [new_nodes[truss[0], 2], new_nodes[truss[1], 2]]
        ax.plot(x, y, z, color='blue', linewidth=8 )
    
    # 绘制边
    for  cable in  cable:
        x = [new_nodes[cable[0], 0], new_nodes[cable[1], 0]]
        y = [new_nodes[cable[0], 1], new_nodes[cable[1], 1]]
        z = [new_nodes[cable[0], 2], new_nodes[cable[1], 2]]
        ax.plot(x, y, z, color='black', linewidth=2)

    #  在每个节点上标注名称
    for i, name in enumerate(node_names):
        ax.text(new_nodes[i, 0],new_nodes[i, 1], new_nodes[i, 2], name, ha='center', va='bottom')
    
    # 设置图形参数
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_box_aspect([1,1,1])  # 设置坐标轴等比例显示

    # 设置展示角度
    ax.view_init(elev=12, azim=146)  # 修改elev和azim来改变展示角度

    # 设置原点位置
    # ax.set_xlim(-40, 120)  # 设置x轴范围
    ax.set_xlim(100, -50)
    ax.set_ylim(100, -50)  # 设置y轴范围
    ax.set_zlim(0, 150)   # 设置z轴范围
   
    plt.pause(0.5)
    # return solution

# 确保模型已加载
model_loaded = load_model('lstm_model.h5')

# 定义机器学习方程
def Ml(R_0, R_get, L_0):
    d = (R_get - R_0) / R_0
    # 采用bending state
    if d < 0:
        # Polynomial equation correction
        y1 = -4.7589 * d**5 - 16.521 * d**4 - 20.239 * d**3 - 9.9675 * d**2 - 0.5464 * d - 0.0016
    else:
        # 机械学习预测
        # 准备输入数据，速度提前告知0.5
        input_data = np.array([[d, 0.5]])
        # 重塑输入数据以符合模型输入 (1, 1, 特征数)
        input_data_reshaped = input_data.reshape(1, 1, input_data.shape[1])
        # 进行预测
        y1 = model_loaded.predict(input_data_reshaped)
        # 从数组中提取单一预测值
        y1 = y1[0][0]

    c = L_0 * (1 + y1)
    return c

# 画图
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
# 设置串口参数
port0 = 'COM3'  # 根据你的 Arduino 连接的串口号进行调整
port1 = 'COM4'
baudrate = 9600  # 根据你的 Arduino 设置的波特率进行调整

# 检查串口是否已关闭
try:
    ser0 = serial.Serial(port0, baudrate)
    ser1 = serial.Serial(port1, baudrate)
    ser0.close()
    ser1.close()
except serial.SerialException:
    print("请关闭串口连接后再运行程序！")
    exit()

# 打开串口连接
ser0 = serial.Serial(port0, baudrate)
ser1 = serial.Serial(port1, baudrate)
time.sleep(5)  # 给Arduino一些时间来重置

# 替换为你的微调模型 API Key
api_key = 'your-fine-tuned-model-api-key'

# 替换为你的微调模型名称
fine_tuned_model_name = "havronl"

def send_command_to_arduino(command):
    """发送指令到Arduino"""
    try:
        ser0.write(command.encode())
        ser1.write(command.encode())
    except serial.SerialException as e:
        print(f"发送到Arduino时发生错误: {str(e)}")

def process_command_with_havronl(user_input):
    """使用havronl处理命令"""
    try:
        response = openai.Completion.create(
            model="fine_tuned_model_name",
            messages=[{"role": "user", "content": user_input}],
            api_key=api_key
        )
        return response.choices[0].message['content'].strip()
    except openai.OpenAIError as e:
        print(f"处理命令时发生错误：{str(e)}")
        return ""

# 用户输入命令
user_command = input("请输入你的命令：")  # Check Tensegrity State

# 使用havronl处理命令
processed_command = process_command_with_havronl(user_command)

# 基于处理结果发送指令到Arduino
if "state" in processed_command.lower():
    send_command_to_arduino('H')
    print('I am checking my shape, here it is:')

    # 读取串口数据
    data0 = ser0.readline().decode('utf-8').rstrip()
    data1 = ser1.readline().decode('utf-8').rstrip()
    lengths = {}

    try:   
        # 从串口读取数据                       
        R_B21C12, R_A11C12, R_A11B21, R_B22C12, R_A21B22, R_A21C12, R_A21C11, R_A21B12, R_B12C11, R_A22C22, R_A22B22, R_B22C22, R_A22B12, R_A22C21, R_B12C21= data0.split(',')
        R_A12C21, R_A12B11, R_B11C21, R_A12B21, R_A12C22, R_B21C22, R_A11B11, R_A11C11, R_B11C11= data1.split(',')

        # 转换数据
        # A11 B21 C12 三角
        R_B21C12 = float(R_B21C12)
        R_A11C12 = float(R_A11C12)
        R_A11B21 = float(R_A11B21)
        # print(R_B21C12,R_A11C12,R_A11B21)
        # 机器学习预测长度 
        L_B21C12 = Ml(R_B21C12_0,R_B21C12,L_B21C12_0)
        L_A11C12 = Ml(R_A11C12_0,R_A11C12,L_A11C12_0)
        L_A11B21 = Ml(R_A11B21_0,R_A11B21,L_A11B21_0)

        # B22 C12 A21                
        R_B22C12 = float(R_B22C12)
        R_A21B22 = float(R_A21B22)
        R_A21C12 = float(R_A21C12)
        # 机器学习预测长度 
        L_B22C12 = Ml(R_B22C12_0,R_B22C12,L_B22C12_0)
        L_A21B22 = Ml(R_A21B22_0,R_A21B22,L_A21B22_0)
        L_A21C12 = Ml(R_A21C12_0,R_A21C12,L_A21C12_0)
        
        # A21, C11, B12
        R_A21C11 = float(R_A21C11)
        R_A21B12 = float(R_A21B12)
        R_B12C11 = float(R_B12C11)
        # 机器学习预测长度 
        L_A21C11 = Ml(R_A21C11_0,R_A21C11,L_A21C11_0)
        L_A21B12 = Ml(R_A21B12_0,R_A21B12,L_A21B12_0)
        L_B12C11 = Ml(R_B12C11_0,R_B12C11,L_B12C11_0)
        
        # A22 C22 B22
        R_A22C22 = float(R_A22C22)
        R_A22B22 = float(R_A22B22)
        R_B22C22 = float(R_B22C22)
        # 机器学习预测长度 
        L_A22C22 = Ml(R_A22C22_0,R_A22C22,L_A22C22_0)
        L_A22B22 = Ml(R_A22B22_0,R_A22B22,L_A22B22_0)
        L_B22C22 = Ml(R_B22C22_0,R_B22C22,L_B22C22_0)
        
        # A22 B12 C21
        R_A22B12 = float(R_A22B12)
        R_A22C21 = float(R_A22C21)
        R_B12C21 = float(R_B12C21)
        # 机器学习预测长度 
        L_A22B12 = Ml(R_A22B12_0,R_A22B12,L_A22B12_0)
        L_A22C21 = Ml(R_A22C21_0,R_A22C21,L_A22C21_0)
        L_B12C21 = Ml(R_B12C21_0,R_B12C21,L_B12C21_0)

        # A12 C21 B11
        R_A12C21 = float(R_A12C21)
        R_A12B11 = float(R_A12B11)
        R_B11C21 = float(R_B11C21)
        # 机器学习预测长度 
        L_A12C21 = Ml(R_A12C21_0,R_A12C21,L_A12C21_0)
        L_A12B11 = Ml(R_A12B11_0,R_A12B11,L_A12B11_0)
        L_B11C21 = Ml(R_B11C21_0,R_B11C21,L_B11C21_0)
        
        # A12 B21 C22 
        R_A12B21 = float(R_A12B21)
        R_A12C22 = float(R_A12C22)
        R_B21C22 = float(R_B21C22)
        # 机器学习预测长度 
        L_A12B21 = Ml(R_A12B21_0,R_A12B21,L_A12B21_0)
        L_A12C22 = Ml(R_A12C22_0,R_A12C22,L_A12C22_0)
        L_B21C22 = Ml(R_B21C22_0,R_B21C22,L_B21C22_0)

        # 这次底部固定 不需要预测
        R_A11B11 = float(R_A11B11)
        R_A11C11 = float(R_A11C11)
        R_B11C11 = float(R_B11C11)              

        lengths = {
            "L_B21C12": L_B21C12,"L_A11C12": L_A11C12,"L_A11B21": L_A11B21,
            "L_B22C12": L_B22C12,"L_A21B22": L_A21B22,"L_A21C12": L_A21C12,
            "L_A21C11": L_A21C11,"L_A21B12": L_A21B12,"L_B12C11": L_B12C11, 
            "L_A22B22": L_A22B22,"L_A22C22": L_A22C22,"L_B22C22": L_B22C22,
            "L_A22B12": L_A22B12,"L_A22C21": L_A22C22,"L_B12C21": L_B12C21,
            "L_A12C21": L_A12C21,"L_A12B11": L_A12B11,"L_B11C21": L_B11C21,              
            "L_A12B21": L_A12B21,"L_A12C22": L_A12C22,"L_B21C22": L_B21C22
        }
        # print("L_B21C12=",L_B21C12,"L_A11C12=",L_A11C12,"L_A11B21=",L_A11B21,
        # " L_B22C12=", L_B22C12,"L_A21B22=",L_A21B22,"L_A21C12=",L_A21C12,
        # " L_A21C11=", L_A21C11,"L_A21B12=",L_A21B12,"L_B12C11=",L_B12C11,                     
        # "L_A22B22=",L_A22B22,"L_A22C22=",L_A22C22,"L_B22C22=", L_B22C22,
        # "L_A22B12=",L_A22B12,"L_A22C21=",L_A22C22,"L_B12C21=", L_B12C21,
        # "L_A12C21=",L_A12C21,"L_A12B11=",L_A12B11,"L_B11C21=", L_B11C21,              
        # "L_A12B21=",L_A12B21,"L_A12C22=",L_A12C22,"L_B21C22=", L_B21C22)

        #绘制图表   更新图表
        update()
        
    except KeyboardInterrupt:
    # 在终止程序前关闭串口
        pass

    except ValueError:
        print("Invalid sensor value")
            
else:
    send_command_to_arduino('L')
    print("无效命令，无法执行。")

time.sleep(5)
# 用户输入命令
user_command1 = input("请输入你的命令：")  # could you tell me the sensors's values 

# 使用havronl处理命令
processed_command1 = process_command_with_havronl(user_command1)

# 基于处理结果发送指令到Arduino
if "values" in processed_command1.lower():
    # send_command_to_arduino('H')
    print('I am checking the sensors values:')
    
    print(f'''
    R_B21C12={R_B21C12} MΩ
    R_A11C12={R_A11C12} MΩ
    R_A11B21={R_A11B21} MΩ
    R_B22C12={R_B22C12} MΩ
    R_A21B22={R_A21B22} MΩ
    R_A21C12={R_A21C12} MΩ
    R_A21C11={R_A21C11} MΩ
    R_A21B12={R_A21B12} MΩ
    R_B12C11={R_B12C11} MΩ
    R_A22C22={R_A22C22} MΩ
    R_A22B22={R_A22B22} MΩ
    R_B22C22={R_B22C22} MΩ
    R_A22B12={R_A22B12} MΩ
    R_A22C21={R_A22C21} MΩ
    R_B12C21={R_B12C21} MΩ
    R_A12C21={R_A12C21} MΩ
    R_A12B11={R_A12B11} MΩ
    R_B11C21={R_B11C21} MΩ
    R_A12B21={R_A12B21} MΩ
    R_A12C22={R_A12C22} MΩ
    R_B21C22={R_B21C22} MΩ
    R_A11B11={R_A11B11} MΩ
    R_A11C11={R_A11C11} MΩ
    R_B11C11={R_B11C11} MΩ
    ''')

else:
    # send_command_to_arduino('L')
    print("无效命令，无法执行。")

time.sleep(5)
# 用户输入命令
user_command2 = input("请输入你的命令：")  # could you tell me the length?

# 使用havronl处理命令
processed_command2 = process_command_with_havronl(user_command2)

# 基于处理结果发送指令到Arduino
if "length" in processed_command2.lower():
    # send_command_to_arduino('H')
    print('I am checking the length (mm)')
    # 打印所有长度
    print("Lengths:", lengths)
else:
    # send_command_to_arduino('L')
    print("无效命令，无法执行。")

# 清理和关闭串行端口
ser0.close()
ser1.close()
